In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [99]:
df = pd.read_csv('/Users/xuetong/Downloads/reviews.csv')

In [100]:
df.head()

,location_id,review_id,source,date,rating
0,4962_221,test_1895327047,GOOGLE_PLACES,10/20/15,1.0
1,4962_221,test_18961550118,FACEBOOK,11/27/17,1.0
2,4962_221,test_2044312531,FACEBOOK,11/15/17,1.0
3,4962_221,test_21399832280,FACEBOOK,7/17/17,1.0
4,4962_221,test_21516225526,FACEBOOK,1/11/16,1.0


In [101]:
df.shape

(179345, 5)

In [102]:
df['date'] = pd.to_datetime(df['date'])

### Problem A
Compare the two location(4962_201 and 4962_380). Which do you think is better? Why?

In [68]:
loc_201 = df[df['location_id'] == '4962_201']
loc_380 = df[df['location_id'] == '4962_380']

In [69]:
loc_201['rating'].describe()

count    926.000000
mean       4.750540
std        0.850675
min        1.000000
25%        5.000000
50%        5.000000
75%        5.000000
max        5.000000
Name: rating, dtype: float64

In [70]:
loc_380['rating'].describe()

count    639.000000
mean       4.669797
std        1.008838
min        1.000000
25%        5.000000
50%        5.000000
75%        5.000000
max        5.000000
Name: rating, dtype: float64

In [71]:
grp_by_source_201 = loc_201.groupby(['source'])['rating'].agg(['mean','count'])

In [72]:
grp_by_source_201

,mean,count
source,,
CARS,4.875862,145
FACEBOOK,4.633333,150
GOOGLE_PLACES,4.749604,631


In [73]:
grp_by_source_380 = loc_380.groupby(['source'])['rating'].agg(['mean','count'])
grp_by_source_380

,mean,count
source,,
CARS,4.909091,11
FACEBOOK,4.547619,252
GOOGLE_PLACES,4.744681,376


In [74]:
# grp_by_date_380 = loc_380.groupby(['date'])['rating'].mean()
# grp_by_date_380.plot()

In [75]:
# grp_by_date_201 = loc_201.groupby(['date'])['rating'].mean()
# grp_by_date_201.plot()

location 4962_201 is a better since mean of rating is higher than that of 4962_380 also it has larger sample size and smaller standard deviation, and the rating from each source is higher.

### Problem B
Compare the two location(4962_381 and 4962_915). Which do you think is better? Why?

In [76]:
loc_381 = df[df['location_id'] == '4962_381']
loc_915 = df[df['location_id'] == '4962_915']

In [77]:
loc_381['rating'].describe()

count    559.000000
mean       4.554562
std        1.131249
min        1.000000
25%        5.000000
50%        5.000000
75%        5.000000
max        5.000000
Name: rating, dtype: float64

In [78]:
loc_915['rating'].describe()

count    521.000000
mean       4.629559
std        1.022351
min        1.000000
25%        5.000000
50%        5.000000
75%        5.000000
max        5.000000
Name: rating, dtype: float64

In [79]:
grp_by_source_381 = loc_381.groupby(['source'])['rating'].agg(['mean','count'])
grp_by_source_381

,mean,count
source,,
CARS,4.740741,27
FACEBOOK,4.518519,486
GOOGLE_PLACES,4.826087,46


In [80]:
grp_by_source_915 = loc_915.groupby(['source'])['rating'].agg(['mean','count'])
grp_by_source_915

,mean,count
source,,
CARS,4.902516,318
FACEBOOK,4.295238,105
GOOGLE_PLACES,4.102041,98


In [81]:
loc_915['date'].nunique()

346

I think location 4962_381 is better.Although the its overall avarage of rating is a little bit lower than that of location 4962_915, but if we look of the segmation of each course location 4962_915 has really low rating in platform facebook and google_places.

### Problem C
create a formula to rank all of the location.

The formula I come up with is Rank_score = AVG(rating) + min(rating_by_source)+ max(count_weight+0.5,1),which considered avarage rating of each location, and minimum of average rating from each sources, as well as how many rate we received from each location, for location has extreme small sample size, there count weight would be smaller. 

In [131]:
groups_loc = df.groupby(['location_id'],as_index = False)['rating'].agg(['mean','count'])

In [132]:
groups_loc_source = df.groupby(['location_id','source'])['rating'].mean().unstack()
groups_loc_source
groups_source_min = groups_loc_source.apply(lambda x: min(x.CARS,x.FACEBOOK,x.GOOGLE_PLACES),axis = 1).to_frame(name = 'min_rate')

In [133]:
groups_source_min.head()

,min_rate
location_id,
4962_10,4.357143
4962_1000,4.080000
4962_1005,3.596154
4962_101,3.672727
4962_103,3.949333


In [110]:
#groups_source_min['location_id'] = groups_source_min.index

In [134]:
groups_source_min.reset_index(inplace= True)
groups_source_min.head()

,location_id,min_rate
0,4962_10,4.357143
1,4962_1000,4.080000
2,4962_1005,3.596154
3,4962_101,3.672727
4,4962_103,3.949333


In [135]:
groups_loc.head()

,mean,count
location_id,,
4962_10,4.410653,582
4962_1000,4.462575,668
4962_1005,4.369085,634
4962_101,4.104953,848
4962_103,4.279449,798


In [136]:
rank = groups_loc.merge(groups_source_min,on = 'location_id')

In [137]:
rank.head()

,location_id,mean,count,min_rate
0,4962_10,4.410653,582,4.357143
1,4962_1000,4.462575,668,4.080000
2,4962_1005,4.369085,634,3.596154
3,4962_101,4.104953,848,3.672727
4,4962_103,4.279449,798,3.949333


In [140]:
np.min(rank['count'])

12

In [141]:
np.max(rank['count'])

1630

In [142]:
rank['count_weight'] = rank['count'].apply(lambda x : x/(np.max(rank['count'])))

In [151]:
rank['rank_rate'] = rank.apply(lambda x: (x['mean'] + x['min_rate'])+max((x['count_weight']+0.5,1)), axis = 1)

In [152]:
rank['rank_result'] = rank['rank_rate'].rank(ascending = False)

In [153]:
rank.sort_values(by = 'rank_result').head(20)

,location_id,mean,count,min_rate,count_weight,rank_rate,rank_result
89,4962_435,4.896817,911,4.875300,0.558896,10.831012,1.0
15,4962_131,4.928571,784,4.845238,0.480982,10.773810,2.0
211,4962_875,4.820546,769,4.808989,0.471779,10.629535,3.0
65,4962_343,4.860262,687,4.741176,0.421472,10.601438,4.0
155,4962_69,4.769674,521,4.762712,0.319632,10.532386,5.0
50,4962_288,4.787415,588,4.730769,0.360736,10.518184,6.0
221,4962_908,4.757447,705,4.702703,0.432515,10.460150,7.0
46,4962_275,4.775304,988,4.575904,0.606135,10.457342,8.0
29,4962_201,4.750540,926,4.633333,0.568098,10.451971,9.0
188,4962_768,4.547239,1630,4.380510,1.000000,10.427750,10.0


### problem 4